In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
%aimport Net
%aimport select_dataset
import select_dataset
import Models 
import Net
from train_test_validat import *
from self_attention import *
import  ecg_get_data 
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

import time
import os

# random_seed = 2
# torch.manual_seed(random_seed)    # reproducible
# torch.cuda.manual_seed_all(random_seed)
# random.seed(random_seed)
# np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

In [2]:
model_path = './model/'+time_str
log_path = './log/'+  time_str
ECG_root = '/workspace/data/Preprocess_HTN/dataII/ECG'
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [3]:
data = select_dataset.splite_dataset('/workspace/data/Preprocess_HTN/dataII/',True)

In [4]:
test_list = data.__get_test_file_list__(True,'外科')
valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.9,True,'外科')  # type: ignore


	
 orginal   fliter department&age
  33342       8479   
	
 orginal   fliter duplicated
   509        423    
	
 orginal   fliter duplicated
   8479       7043   
	
       HTN  NHTN 
test   423   423 
	
 orginal   fliter department&age
  65932      11329   
	
 orginal   fliter duplicated
   1354       1111   
	
 orginal   fliter duplicated
  11329       9216   
lack sample like : Pandas(Index=1068, num=3257, name='史元昌', years=98, gender='男', department='干部保健二科三区', diagnose='高血压', ID='271758', date='2020-11-20 16:04:58', ECG_path='20-3257_100-1_HTN.npy')
lack sample like : Pandas(Index=296, num=369, name='李文友', years=95, gender='男', department='', diagnose='', ID='', date='', ECG_path='00-369_98-1_HTN.npy')
lack sample like : Pandas(Index=753, num=13411, name='李如绥', years=95, gender='男', department='干部保健二科一区', diagnose='高血压病3级（极高危）', ID='297125', date='2020-10-11 16:33:39', ECG_path='20-13411_97-1_HTN.npy')
	
       HTN  NHTN 
train  999   999 
valid  112   112 
 add    0   8105 


In [5]:
test_Dataset = ecg_get_data.ECG_Dataset(ECG_root,test_list,EcgChannles_num,EcgLength_num)
valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)
train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,shadow_npy_folder=ECG_root,shadow_npy_files_list=addition_train_list)

npys:{%d} 846
npys:{%d} 224
npys:{%d} 1998
shadow_npys:{%d} 8105


In [6]:
BATCH_SIZE = 128

FOLDS = 1
EPOCHS = 500  
PATIENCE = 100
LR = 0.01

In [7]:
torch.cuda.empty_cache()# 清空显卡cuda
# NET = [
#     Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25)
# ]
# NET = [Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore
NET = [Net.MLBFNet(True,res = True,se = True,Dropout_rate = 0.3) ] # type: ignore

In [8]:
from torch.utils.tensorboard import SummaryWriter
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
# writer.add_graph(NET[0], torch.zeros((1,12,5000)))  #模型及模型输入数 据
torch.cuda.empty_cache()# 清空显卡cuda

In [9]:
import torch.nn.functional as F

def linear_combination(x, y, epsilon): 
    return epsilon*x + (1-epsilon)*y
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon:float=0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
    
    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return linear_combination(loss/n, nll, self.epsilon)
    
class GCELoss(nn.Module):
    def __init__(self, num_classes=2, q=0.7):
        super(GCELoss, self).__init__()
        self.q = q
        self.num_classes = num_classes

    def forward(self, pred, labels):
        pred = torch.nn.functional.softmax(pred, dim=1)
        pred = torch.clamp(pred, min=1e-4, max=1.0)
        label_one_hot = torch.nn.functional.one_hot(labels, self.num_classes).float().to(pred.device)
        loss = (1. - torch.pow(torch.sum(label_one_hot * pred, dim=1), self.q)) / self.q
        return loss.mean()


In [10]:
import math
for fold in range(FOLDS):
    #每个人fold都重新抽取
    NET[fold].to(DEVICE)
    test_list = data.__get_test_file_list__(True,'外科')
    valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.9,True,'外科')  # type: ignore
        
    train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,shadow_npy_folder=ECG_root,shadow_npy_files_list=addition_train_list)
    test_Dataset = ecg_get_data.ECG_Dataset(ECG_root,test_list,EcgChannles_num,EcgLength_num)
    valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)

    train_dataloader = Data.DataLoader(dataset=train_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4,pin_memory=True)
    valid_dataloader = Data.DataLoader(dataset=valid_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4,pin_memory=True)
    test_dataloader = Data.DataLoader(dataset=test_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4,pin_memory=True)
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0, positive=False)
    optimizer  = torch.optim.AdamW(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    
    warm_up_iter = 10
    T_max = 500	# 周期
    lr_max = 1e-3	# 最大值
    lr_min = 1e-5	# 最小值
    lambda0 = lambda cur_iter: lr_min if  cur_iter < warm_up_iter else \
        (lr_min + 0.5*(lr_max-lr_min)*(1.0+math.cos( (cur_iter-warm_up_iter)/(T_max-warm_up_iter)*math.pi)))/0.01
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda0)
    criterion = torch.nn.CrossEntropyLoss()   
    # criterion = LabelSmoothingCrossEntropy()
    
    best_test_F1 = 0
    for epoch in range(1,EPOCHS):
        #每个epoch 更新train中的NHTN类 大概13s
        time_all=0
        start_time = time.time()
        #y_true,y_pred,train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # type: ignore # 训练模型
        y_true,y_pred,train_loss,train_acc = train_model_mixup(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # type: ignore # 训练模型
        time_all = time.time()-start_time
        F1_score_train =f1_score(y_true, y_pred, average='macro')#F1分数
        C0 = confusion_matrix(y_true,y_pred)
        y_true,y_pred,validate_loss,validate_acc = eval_model(valid_dataloader,criterion,NET[fold],DEVICE) # 验证模型
        F1_score_valid =f1_score(y_true, y_pred, average='macro')#F1分数
        C1 = confusion_matrix(y_true,y_pred)
        y_true,y_pred,test_loss,test_acc = eval_model(test_dataloader,criterion,NET[fold],DEVICE) # 验证模型
        F1_score_test =f1_score(y_true, y_pred, average='macro')#F1分数
        C2 = confusion_matrix(y_true,y_pred)

        # writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        # writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        # writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)
        # writer.add_scalars(main_tag=str(fold)+'_F1_score',tag_scalar_dict={'train':F1_score_train,'validate': F1_score_valid},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss,'test':test_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc,'test':test_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_F1_score',tag_scalar_dict={'train':F1_score_train,'validate': F1_score_valid,'test':F1_score_test},global_step=epoch)        
        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - F1 score: %.5f - Val_loss: %.5f - Val_acc: %.5f - F1 score: %.5f - T_Time: %.5f' %(epoch,train_loss,train_acc,F1_score_train,validate_loss,validate_acc,F1_score_valid,time_all))
        print('当前学习率：%.8f' %optimizer.state_dict()['param_groups'][0]['lr'])
        print('train:\n',C0)
        print('validate:\n',C1)
        print('test:\n',C2)
        
        if(F1_score_test>best_test_F1):
            best_test_F1 = F1_score_test
            torch.save(NET[fold].state_dict(), model_path+'/parameter_best_test_' + str(fold) + '.pt')
        
        scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold],fold)):
            print("Early stopping")
            break
        if(epoch>T_max):
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

	
 orginal   fliter department&age
  33342       8479   
	
 orginal   fliter duplicated
   509        423    
	
 orginal   fliter duplicated
   8479       7043   
	
       HTN  NHTN 
test   423   423 
	
 orginal   fliter department&age
  65932      11329   
	
 orginal   fliter duplicated
   1354       1111   
	
 orginal   fliter duplicated
  11329       9216   
lack sample like : Pandas(Index=1068, num=3257, name='史元昌', years=98, gender='男', department='干部保健二科三区', diagnose='高血压', ID='271758', date='2020-11-20 16:04:58', ECG_path='20-3257_100-1_HTN.npy')
	
       HTN  NHTN 
train  999   999 
valid  112   112 
 add    0   8105 
npys:{%d} 1998
shadow_npys:{%d} 8105
npys:{%d} 846
npys:{%d} 224
- Epoch: 1 - Train_loss: 0.69492 - Train_acc: 0.51301 - F1 score: 0.49961 - Val_loss: 0.69307 - Val_acc: 0.50781 - F1 score: 0.33333 - T_Time: 37.18632
当前学习率：0.00000010
train:
 [[349 650]
 [323 676]]
validate:
 [[  0 112]
 [  0 112]]
test:
 [[  0 423]
 [  0 423]]
Validation F1 score  increase to (inf

KeyboardInterrupt: 